# Installation Instructions

Clone the repos, install dependencies and install packages terrapin, quest and gsshapy.

Note: terrapin isn't used in this example but it will be in future examples...

```bash
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/quest.git
git clone git@public.git.erdc.dren.mil:computational-analysis-and-mechanics/terrapin.git
git clone git@github.com:CI-WATER/gsshapy.git

conda env create -n earthsim -f ./terrapin/py3_conda_environment.yml
conda env update -n earthsim -f ./quest/py3_conda_environment.yml
conda env update -n earthsim -f ./gsshapy/conda_env.yml
source activate earthsim

cd terrapin
python setup.py develop
cd ../quest
python setup.py develop
cd ../gsshapy
python setup.py develop
```


In [ ]:
from datetime import datetime, timedelta
import os

import geopandas as gpd
from gsshapy.modeling import GSSHAModel
import quest

Function to automate the downloading process:

In [ ]:
def download_data(service_uri, bounds, collection_name):
    """
    Downloads raster data from source uri and adds to a quest collection.
    
    If multiple raster tiles are retrieved for the given bounds it calls a quest 
    filter to merge the tiles into a single raster.
    """
    
    # download the features (i.e. locations) and metadata for the given web service
    # the first time this is run it will take longer since it downloads 
    # and caches all metadata for the given service.
    features = quest.api.get_features(uris=service_uri,
                                      filters={'bbox': bounds},
                                      as_dataframe=True,
                                      )
    
    # add the selected features to a quest collection
    added_features = quest.api.add_features(collection_name, features)
    
    # stage the data for download, optional parameters can be provided here 
    # for certain web services (i.e. date range etc), raster services don't 
    # typically have any optional parameters.
    staged_datasets = quest.api.stage_for_download(uris=added_features)
    
    # download the staged datasets
    quest.api.download_datasets(datasets=staged_datasets)
    final_datasets = staged_datasets
    
    # if more than one raster tile downloaded, merge into a single raster tile
    if len(staged_datasets) > 1:
        merged_datasets = quest.api.apply_filter(name='raster-merge',
                                                 datasets=staged_datasets)
        final_datasets = merged_datasets['datasets']
        # delete the original individual tiles
        quest.api.delete(staged_datasets)

    # return file path of final geotiff
    downloaded_datasets = quest.api.get_metadata(final_datasets)
    downloaded_file_paths = [dataset['file_path'] for dataset
                             in downloaded_datasets.values()]
    return downloaded_file_paths[0]

Parameters to change for the run:

In [ ]:
elevation_service = 'svc://usgs-ned:13-arc-second'
land_use_service = 'svc://usgs-nlcd:2011'
land_use_grid_id = 'nlcd'
base_dir = os.getcwd() # path needs to be absolute for gsshapy
boundary_shapefile = os.path.join(base_dir, 'vicksburg_watershed', 'watershed_boundary.shp')
gssha_model_name = 'vicksburg_south'
gssha_model_directory = os.path.join(base_dir, gssha_model_name)

Get the download boundary from the shapefile:

In [ ]:
boundary_gpd = gpd.read_file(boundary_shapefile)
bounds = [str(x) for x in boundary_gpd.geometry.bounds.values[0]]

The collection is where we will origanize the downloaded data.

In [ ]:
try:
    quest.api.new_collection(gssha_model_name)
except ValueError as e:
    print(e)

Download the data. This may take some time.

In [ ]:
elevation_file_path = download_data(elevation_service, bounds, gssha_model_name)

In [ ]:
land_use_file_path = download_data(land_use_service, bounds, gssha_model_name)

Use the data to generate a GSSHA model. This may take a minute.

In [ ]:
# make the directory for the output
try:
    os.mkdir(gssha_model_directory)
except OSError:
    pass

# generate GSSHA model files
model = GSSHAModel(project_name=gssha_model_name,
                   project_directory=gssha_model_directory,
                   mask_shapefile=boundary_shapefile,
                   elevation_grid_path=elevation_file_path,
                   land_use_grid=land_use_file_path,
                   land_use_grid_id=land_use_grid_id,
                   out_hydrograph_write_frequency=1,
                   roughness=0.04,
                   )

# add card for max depth
model.project_manager.setCard('FLOOD_GRID',
                              '{0}.fgd'.format(gssha_model_name),
                              add_quotes=True)
# TODO: Add depth grids to simulation
# MAP_FREQ, DEPTH

# add event for simulation
model.set_event(simulation_start=datetime.utcnow(),
                simulation_duration=timedelta(seconds=2*60),
                rain_intensity=24,
                rain_duration=timedelta(seconds=1*60),
                )
model.write()

In [14]:
help(GSSHAModel)

Help on class GSSHAModel in module gsshapy.modeling.model:

class GSSHAModel(builtins.object)
 |  This class manages the generation and modification of
 |  models for GSSHA.
 |  
 |  Parameters:
 |       project_directory(str): Directory to write GSSHA project files to.
 |       project_name(Optional[str]): Name of GSSHA project. Required for new model.
 |       mask_shapefile(Optional[str]): Path to watershed boundary shapefile. Required for new model.
 |       grid_cell_size(Optional[str]): Cell size of model (meters). Required for new model.
 |       elevation_grid_path(Optional[str]): Path to elevation raster used for GSSHA grid. Required for new model.
 |       simulation_timestep(Optional[float]): Overall model timestep (seconds). Sets TIMESTEP card. Required for new model.
 |       out_hydrograph_write_frequency(Optional[str]): Frequency of writing to hydrograph (minutes). Sets HYD_FREQ card. Required for new model.
 |       roughness(Optional[float]): Value of uniform manning's